In [1]:
import getpass
import os
import langchain

from langchain.schema.messages import HumanMessage, SystemMessage,AIMessage

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint="https://12205-m2hl4tqk-eastus.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-08-01-preview",
    azure_deployment="gpt-35-turbo",
    openai_api_version="2024-08-01-preview",
    api_key="d7f27353b3b3463bb02b2708df922f35",  
)

In [3]:
messages = [
    SystemMessage(content="用中文回复"),
    HumanMessage(content="中国有几个省份"),
  
]
model.invoke(messages)

AIMessage(content='中国共有34个省级行政区，其中包括23个省、5个自治区、4个直辖市和2个特别行政区。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 25, 'total_tokens': 64, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-cdedd16b-478c-4b28-b325-696a155c016b-0', usage_met

# 输出解析器
导入简单的输出解析器，从而得到只想处理字符串响应

In [4]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
   SystemMessage(content="用中文回复"),
    HumanMessage(content="中国有几个省份"),
]
result=model.invoke(messages)
parser.invoke(result)

'中国共有34个省级行政区，包括23个省、5个自治区、4个直辖市和2个特别行政区。'

#### 可以将模型与此输出解析器进行“链式”连接，即每次都会调用此输出解析器。此链采用语言模型的输入类型（字符串或消息列表）并返回输出解析器的输出类型（字符串），如下

In [6]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
   SystemMessage(content="用中文回复"),
    HumanMessage(content="中国有几个省份"),
]
chain = model | parser
chain.invoke(messages)

'目前中国共有34个省级行政区，包括23个省、5个自治区、4个直辖市和2个特别行政区。'

# 提示词模板

PromptTemplates是LangChain 中的一个概念，旨在直接将消息列表传递给语言模型。PromptTemplates通常是由用户输入和应用逻辑的组合构建而成的。这个应用逻辑通常会将原始用户输入转换为准备传递给语言模型的消息列表，常见的转换包括添加系统消息或使用用户输入格式化模板

### 创建一个 PromptTemplate，它将接收两个用户变量：
language: 要翻译成的语言

text: 要翻译的文本

In [11]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "eglish", "text": "hi"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into eglish:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

### 可以看到它返回的是一个ChatPromptValue，由两个消息组成。如果想直接访问这些消息：

In [12]:
result.to_messages()
[SystemMessage(content='Translate the following into english:'), HumanMessage(content='hi')]

[SystemMessage(content='Translate the following into english:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

# 使用LCEL连接组件

### 可以使用 | 操作符将其与上面的模型和输出解析器结合：

In [18]:
chain = prompt_template | model | parser

In [21]:
chain.invoke({"language": "eglish", "text": "你好"})

'Hello.'

# 构建聊天机器人

### 构建个聊天机器人，仅使用语言模型进行对话

对话式RAG: 在外部数据源上启用聊天机器人体验

代理: 构建一个可以采取行动的聊天机器人

In [23]:
import getpass
import os
import langchain

from langchain.schema.messages import HumanMessage, SystemMessage,AIMessage

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint="https://12205-m2hl4tqk-eastus.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-08-01-preview",
    azure_deployment="gpt-35-turbo",
    openai_api_version="2024-08-01-preview",
    api_key="d7f27353b3b3463bb02b2708df922f35",  
)

直接使用模型，ChatModel是LangChain的运行接口实例，它提供了一个标准接口供我们与之交互

In [26]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
from langchain_core.messages import HumanMessage
result=model.invoke([HumanMessage(content="My name is kitty")])
parser.invoke(result)

'Hello Kitty! How can I assist you today?'

继续提问

In [31]:
result=model.invoke([HumanMessage(content="What's my name?")])
parser.invoke(result)

"I'm sorry, as an AI language model, I don't have access to your personal information such as your name."

很明显，并未与之前的对话建立上下文的联系，从而无法回答问题。那么将整个对话历史传递给模型，会得到：

In [33]:
from langchain_core.messages import AIMessage
result=model.invoke(
    [
        HumanMessage(content="Hi! I'm kitty"),
        AIMessage(content="Hello kitty! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)
parser.invoke(result)

'Your name is kitty.'

### 可以使用消息历史类来包装模型，使其具有状态。通过跟踪模型的输入和输出，将其存储在某个数据存储中。 未来的交互将加载这些消息，并将其作为输入的一部分传递给链。 

In [34]:
%pip install langchain_community

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simpleNote: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
     -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
     ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
     -------------------- ------------------- 1.3/2.5 MB 1.8 MB/s eta 0:00:01
     ------------------------- -------------- 1.6/2.5 MB 1.7 MB/s eta 0:00:01
     --------------------------------- ------ 2.1/2.5 MB 1.8 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 1.8 MB/s eta 0:00:00


导入相关类并设置链，该链包装模型并添加此消息历史。一个关键部分是我们作为 get_session_history 传入的函数，这个函数预计接受一个 session_id 并返回一个消息历史对象，这个 session_id用于区分不同的对话，并应作为配置的一部分在调用新链时传入

In [35]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [36]:
config = {"configurable": {"session_id": "aaa"}}

In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm kitty")],
    config=config,
)
response.content

'Hello Kitty! How can I assist you today?'

In [39]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Kitty, according to what you told me earlier.'

更改配置以引用不同的session_id，可以看到它开始新的对话

In [40]:
config = {"configurable": {"session_id": "bbb"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

"I'm sorry, as an AI language model, I don't have access to your personal information such as your name."

## 提示词模板

#### 提示词模板可以将原始用户信息转换为大型语言模型可以处理的格式。在这种情况下，原始用户输入只是一个消息，我们将其传递给大型语言模型。比如添加一个带有一些自定义指令的系统消息（但仍然将消息作为输入）并且添加除了消息之外的更多输入。首先添加一个系统消息，我们将创建一个 ChatPromptTemplate，并且将利用 MessagesPlaceholder 来传递所有消息

In [41]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [42]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm kitty")]})
response.content

'Hello Kitty! How can I assist you today?'

将其包装在与之前相同的消息历史对象中：

In [43]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "ccc"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Tom")],
    config=config,
)
response.content

'Hello Tom! How can I assist you today?'

In [44]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Tom.'

再复杂一些

In [45]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [46]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Korean"}
)
response.content

'안녕하세요, 밥님! 어떻게 도와드릴까요?'

将这个更复杂的链封装在一个消息历史类中。由于输入中有多个键，需要指定正确的键来保存聊天历史

In [47]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "ddd"}}

In [48]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)
response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [49]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)
response.content

'Tu nombre es Todd.'

## 管理对话历史

LangChain提供了一些内置的助手来管理消息列表。在这种情况下，使用trim_messages助手来减少发送给模型的消息数量。修剪器允许指定希望保留的令牌数量，以及其他参数，例如是否希望始终保留系统消息以及是否允许部分消息：

In [93]:
from typing import List
# pip install tiktoken
import tiktoken
from langchain_core.messages import BaseMessage, ToolMessage
def str_token_counter(text: str) -> int:
    enc = tiktoken.get_encoding("o200k_base")
    return len(enc.encode(text))


def tiktoken_counter(messages: List[BaseMessage]) -> int:
    """Approximately reproduce https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

    For simplicity only supports str Message.contents.
    """
    num_tokens = 3  # every reply is primed with <|start|>assistant<|message|>
    tokens_per_message = 3
    tokens_per_name = 1
    for msg in messages:
        if isinstance(msg, HumanMessage):
            role = "user"
        elif isinstance(msg, AIMessage):
            role = "assistant"
        elif isinstance(msg, ToolMessage):
            role = "tool"
        elif isinstance(msg, SystemMessage):
            role = "system"
        else:
            raise ValueError(f"Unsupported messages type {msg.__class__}")
        num_tokens += (
            tokens_per_message
            + str_token_counter(role)
            + str_token_counter(msg.content)
        )
        if msg.name:
            num_tokens += tokens_per_name + str_token_counter(msg.name)
    return num_tokens


trim_messages(
    messages,
    max_tokens=45,
    strategy="last",
    token_counter=tiktoken_counter,
)

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [94]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=tiktoken_counter,
    include_system=True,
    allow_partial=False,
    start_on="human",
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

在链中使用它，我们只需在将 messages 输入传递给提示之前运行修剪器，现在如果尝试询问模型我的名字，它将不知道，因为我们修剪了聊天历史的那部分：

In [95]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "Chinese",
    }
)
response.content

"I'm sorry, but I don't know your name."

如果我们询问最近几条消息中的信息，那么它会记住：

In [98]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "Eglish",
    }
)
response.content

'You asked "what\'s 2 + 2"'

现在将其包装在消息历史中：

In [99]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "fff"}}

In [100]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)
response.content

"I'm sorry, but I don't know your name as you haven't told me yet."

声明姓名的第一条消息已被删除。此外，聊天历史中现在有两条新消息（我们最新的问题和最新的回答），这意味着以前可以在我们的对话历史中访问的更多信息现在不再可用。在这种情况下，最初的数学问题也已从历史中删除，因此模型不再知道它：

In [101]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)
response.content

"I'm sorry, but I don't have the ability to access your previous conversation history. Can you please remind me of the math problem you asked?"

## 流式处理

对于聊天机器人应用程序来说，流式处理很重要。由于大型语言模型有时可能需要一段时间才能响应，因此为了改善用户体验，大多数应用程序所做的一件事是随着每个令牌的生成流回，这样用户就可以看到进度。所有链都暴露一个.stream 方法，使用消息历史的链也不例外，我们可以使用该方法获取流式响应：

In [104]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm kitty. tell me a new")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

||Hi|,| Kitty|!| Do| you| mean| "|tell| me| a| news|"?| If| so|,| I| can| provide| you| with| a| recent| news| headline|:

|"|NASA|'s| Per|se|ver|ance| rover| successfully| lands| on| Mars|,| begins| mission| to| search| for| signs| of| ancient| life|"||

# 向量存储和检索器

## 文档

#### LangChain 实现了一个 文档 抽象，旨在表示一个文本单元及其相关元数据。它有两个属性：


page_content：一个表示内容的字符串

metadata：一个包含任意元数据的字典，metadata属性可以捕获有关文档来源、与其他文档的关系以及其他信息

生成了五个文档，包含指示三个不同“来源”的元数据：

In [106]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

## 向量存储